Q1  Discuss the scenarios where multithreading is preferable to multiprocessing and scenarios where
multiprocessing is a better choice ?

ANS. When it comes to performance, the choice between multithreading and multiprocessing often depends on the nature of the task at hand:

1. Multithreading:

- I/O-bound tasks: Multithreading is preferable when dealing with I/O-bound tasks like file operations, web scraping, or network requests. These tasks spend a lot of time waiting for external resources, so having multiple threads can keep the CPU busy while waiting.

- Shared memory space: If tasks need to share a lot of data or resources, multithreading can be more efficient because threads in the same process can share memory space easily.

- Lower overhead: Creating and managing threads has less overhead compared to processes, making it suitable for lightweight tasks.

2. Multiprocessing:

- CPU-bound tasks: Multiprocessing is better for CPU-bound tasks where processes are heavily using the CPU, like complex calculations, data processing, or machine learning. Each process can run on a different CPU core, providing true parallelism.

- Process isolation: If tasks need to be isolated from each other due to resource contention or security reasons, multiprocessing is the way to go. Each process runs in its own memory space, reducing the risk of data corruption.

- Bypassing GIL: In Python, the Global Interpreter Lock (GIL) can be a bottleneck for CPU-bound tasks in multithreading. Multiprocessing spawns separate Python interpreter processes, allowing you to bypass the GIL and make full use of multiple cores.

Q2 Describe what a process pool is and how it helps in managing multiple processes efficiently ?

ANS. A process pool is a collection of worker processes that are managed by a pool manager. Here's how it works and why it’s beneficial:

- How It Works:

1. Pooling Resources: A process pool creates a fixed number of processes in advance and keeps them ready to handle tasks.

2. Task Assignment: When a task is submitted, it is assigned to an available process in the pool.

3. Reusing Processes: Once a process completes its task, it becomes available for new tasks, avoiding the overhead of creating and destroying processes repeatedly.

- Benefits:

- Efficient Resource Management: Keeps a steady number of processes, reducing the overhead of process creation and destruction.

- Load Balancing: Distributes tasks across multiple processes, ensuring efficient use of system resources.

- Simplified Code: Abstracts the complexity of process management, allowing developers to focus on the tasks themselves rather than handling the process lifecycle.

- Scalability: Easily scales to handle varying workloads by adjusting the number of processes in the pool.

Q3  Explain what multiprocessing is and why it is used in Python programs?

ANS. Multiprocessing in Python is a technique that allows the execution of multiple processes simultaneously, enabling parallelism. Each process runs independently and has its own memory space, unlike threads which share the same memory.

**Why It’s Used in Python Programs:**

- Bypass the GIL: Python's Global Interpreter Lock (GIL) can be a bottleneck for CPU-bound tasks in multithreading. Multiprocessing spawns separate Python interpreter processes, allowing full utilization of multiple CPU cores and bypassing the GIL.

- True Parallelism: It enables true parallel execution of tasks, making it ideal for computationally intensive operations, such as complex calculations, data processing, and machine learning tasks.

- Improved Performance: By dividing tasks into multiple processes, you can significantly reduce execution time and improve the performance of your applications.

- Isolation: Each process runs in its own memory space, reducing the risk of data corruption and making it easier to manage resources independently.

Q4  Write a Python program using multithreading where one thread adds numbers to a list, and another
thread removes numbers from the list. Implement a mechanism to avoid race conditions using
threading.Lock?

In [ ]:
import threading
import time
import random

# Shared list
numbers = []

# Create a lock object
lock = threading.Lock()

def add_numbers():
    for _ in range(10):
        lock.acquire()
        try:
            num = random.randint(1, 100)
            numbers.append(num)
            print(f"Added {num} to the list.")
        finally:
            lock.release()
        time.sleep(random.uniform(0.1, 0.5))

def remove_numbers():
    for _ in range(10):
        lock.acquire()
        try:
            if numbers:
                num = numbers.pop(0)
                print(f"Removed {num} from the list.")
        finally:
            lock.release()
        time.sleep(random.uniform(0.1, 0.5))

# Create threads
t1 = threading.Thread(target=add_numbers)
t2 = threading.Thread(target=remove_numbers)

# Start threads
t1.start()
t2.start()

# Wait for both threads to complete
t1.join()
t2.join()

print("Final list:", numbers)


Added 7 to the list.
Removed 7 from the list.
Added 4 to the list.
Removed 4 from the list.
Added 99 to the list.
Added 84 to the list.
Removed 99 from the list.
Added 52 to the list.
Removed 84 from the list.
Removed 52 from the list.
Added 25 to the list.
Removed 25 from the list.
Added 41 to the list.
Removed 41 from the list.
Added 74 to the list.
Removed 74 from the list.
Added 76 to the list.
Added 20 to the list.
Final list: [76, 20]


Q5  Describe the methods and tools available in Python for safely sharing data between threads and
processes?

ANS. Given multiple threads in the program and one wants to safely communicate or exchange data between them.

Perhaps the safest way to send data from one thread to another is to use a Queue from the queue library. To do this, create a Queue instance that is shared by the threads. Threads then use put() or get() operations to add or remove items from the queue as shown in the code given below.

Code #1 :

In [ ]:
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
	while True:
		# Produce some data
		...
		out_q.put(data)

# A thread that consumes data
def consumer(in_q):
	while True:
		# Get some data
		data = in_q.get()
		# Process the data
		...

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target = consumer, args =(q, ))
t2 = Thread(target = producer, args =(q, ))
t1.start()
t2.start()


Exception in thread Thread-11 (producer):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-72ee7c141c3d>", line 9, in producer
NameError: name 'data' is not defined


Queue instances already have all of the required locking, so they can be safely shared by as many threads as per requirement. When using queues, it can be somewhat tricky to coordinate the shutdown of the producer and consumer.

A common solution to this problem is to rely on a special sentinel value, which when placed in the queue, causes consumers to terminate as shown in the code below:

Code #2 :

In [ ]:
from queue import Queue
from threading import Thread

# Object that signals shutdown
_sentinel = object()

# A thread that produces data
def producer(out_q):
	while running:
		# Produce some data
		...
		out_q.put(data)

	# Put the sentinel on the queue to indicate completion
	out_q.put(_sentinel)


# A thread that consumes data
def consumer(in_q):
	while True:
		# Get some data
		data = in_q.get()

		# Check for termination
		if data is _sentinel:
			in_q.put(_sentinel)
			break
		...


A subtle feature of the code above is that the consumer, upon receiving the special sentinel value, immediately places it back onto the queue. This propagates the sentinel to other consumers threads that might be listening on the same queue—thus shutting them all down one after the other.

Although queues are the most common thread communication mechanism, one can build own data structures as long as one adds the required locking and synchronization. The most common way to do this is to wrap your data structures with a condition variable.

Code #3 : Building a thread-safe priority queue

In [ ]:
import heapq
import threading

class PriorityQueue:
	def __init__(self):
		self._queue = []
		self._count = 0
		self._cv = threading.Condition()

	def put(self, item, priority):
		with self._cv:
			heapq.heappush(self._queue, (-priority, self._count, item))
			self._count += 1
			self._cv.notify()

	def get(self):
		with self._cv:
			while len(self._queue) == 0:
				self._cv.wait()
			return heapq.heappop(self._queue)[-1]


Thread communication with a queue is a one-way and non-deterministic process. In general, there is no way to know when the receiving thread has actually received a message and worked on it. However, Queue objects do provide some basic completion features, as illustrated by the task_done() and join() methods in the example given below –

Code #4 :

In [ ]:
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
	while running:
		# Produce some data
		...
		out_q.put(data)

# A thread that consumes data
def consumer(in_q):
	while True:
		# Get some data
		data = in_q.get()
		# Process the data
		...
		# Indicate completion
		in_q.task_done()

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target = consumer, args =(q, ))
t2 = Thread(target = producer, args =(q, ))
t1.start()
t2.start()

# Wait for all produced items to be consumed
q.join()


Exception in thread Thread-13 (producer):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-5-aba1feb56bee>", line 6, in producer
NameError: name 'running' is not defined


Q6  Discuss why it’s crucial to handle exceptions in concurrent programs and the techniques available for
doing so ?

ANS. **Handling exceptions in concurrent programs is crucial for several reasons**:

**Why It's Crucial:**

1. Stability: Without proper exception handling, an unhandled exception in one thread or process can cause the entire program to crash, leading to unexpected downtime and potential data loss.

2. Debugging: Proper exception handling helps in debugging by capturing the exception details, making it easier to diagnose and fix issues.

3. Resource Management: Unhandled exceptions can lead to resource leaks (e.g., open files, network connections) if resources aren't properly released.

4. User Experience: For user-facing applications, exceptions need to be handled gracefully to avoid abrupt crashes and provide meaningful error messages or fallback mechanisms.

**Techniques Available:**

Try-Except Blocks:

Use try-except blocks to catch and handle exceptions.

Example:

In [ ]:
a = [1, 2, 3]
try:
    print ("Second element = %d" %(a[1]))

    print ("Fourth element = %d" %(a[3]))

except:
    print ("An error occurred")


Second element = 2
An error occurred


**Try with Else Clause**

In Python, you can also use the else clause on the try-except block which must be present after all the except clauses. The code enters the else block only if the try clause does not raise an exception.

Try with else clause

The code defines a function AbyB(a, b) that calculates c as ((a+b) / (a-b)) and handles a potential ZeroDivisionError. It prints the result if there’s no division by zero error. Calling AbyB(2.0, 3.0) calculates and prints -5.0, while calling AbyB(3.0, 3.0) attempts to divide by zero, resulting in a ZeroDivisionError, which is caught and “a/b results in 0” is printed.

In [1]:
def AbyB(a , b):
    try:
        c = ((a+b) / (a-b))
    except ZeroDivisionError:
        print ("a/b result in 0")
    else:
        print (c)
AbyB(2.0, 3.0)
AbyB(3.0, 3.0)


-5.0
a/b result in 0


**Finally Keyword in Python**

Python provides a keyword finally, which is always executed after the try and except blocks. The final block always executes after the normal termination of the try block or after the try block terminates due to some exception. The code within the finally block is always executed.

In [2]:
try:
    k = 5//0
    print(k)

except ZeroDivisionError:
    print("Can't divide by zero")

finally:
    print('This is always executed')


Can't divide by zero
This is always executed


**Raising Exception**

The raise statement allows the programmer to force a specific exception to occur. The sole argument in raise indicates the exception to be raised. This must be either an exception instance or an exception class (a class that derives from Exception).

In [4]:
try:
  raise NameError("Hi there")
except NameError:
    print ("An exception")
    raise

An exception


NameError: Hi there

The output of the above code will simply line printed as “An exception” but a Runtime error will also occur in the last due to the raise statement in the last line.

Q7  Create a program that uses a thread pool to calculate the factorial of numbers from 1 to 10 concurrently.
Use concurrent.futures.ThreadPoolExecutor to manage the threads ?

In [5]:
import concurrent.futures
import math

# Function to calculate factorial
def factorial(n):
    return math.factorial(n)

# List of numbers to calculate factorial
numbers = list(range(1, 11))

# Using ThreadPoolExecutor to manage threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(factorial, numbers)

# Display the results
for number, result in zip(numbers, results):
    print(f"Factorial of {number} is {result}")


Factorial of 1 is 1
Factorial of 2 is 2
Factorial of 3 is 6
Factorial of 4 is 24
Factorial of 5 is 120
Factorial of 6 is 720
Factorial of 7 is 5040
Factorial of 8 is 40320
Factorial of 9 is 362880
Factorial of 10 is 3628800


 Q8 Create a Python program that uses multiprocessing.Pool to compute the square of numbers from 1 to 10 in
parallel. Measure the time taken to perform this computation using a pool of different sizes (e.g., 2, 4, 8
processes) ?

In [6]:
import multiprocessing
import time

# Function to compute square
def compute_square(n):
    return n * n

# List of numbers to compute squares
numbers = list(range(1, 11))

# Function to measure time taken with different pool sizes
def measure_time(pool_size):
    with multiprocessing.Pool(processes=pool_size) as pool:
        start_time = time.time()
        results = pool.map(compute_square, numbers)
        end_time = time.time()
        duration = end_time - start_time
    return duration, results

# Measure time taken for pool sizes 2, 4, and 8
for pool_size in [2, 4, 8]:
    duration, results = measure_time(pool_size)
    print(f"Pool size: {pool_size}")
    print(f"Time taken: {duration:.4f} seconds")
    print(f"Results: {results}\n")


Pool size: 2
Time taken: 0.0025 seconds
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

Pool size: 4
Time taken: 0.0043 seconds
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

Pool size: 8
Time taken: 0.0028 seconds
Results: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

